In [1]:
import pandas as pd
import numpy as np
import json
import requests
import xmltodict
import time
import pickle
import os
import sqlalchemy
import urllib.request
from mutagen.mp3 import MP3

In [2]:
album_art = 'https://s3.amazonaws.com/dataskeptic.com/img/2018/fake-news/fake-news-album-400.jpg'

In [3]:
config = json.load(open('../config.json', 'r'))

In [4]:
user = config['db']['username']
password = config['db']['password']
host = config['db']['host']
port = config['db']['port']
dbname = config['db']['dbname']

In [5]:
template = "mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}"
connection_string = template.format(user=user, password=password, host=host, port=port, dbname=dbname)

In [6]:
conn = sqlalchemy.create_engine(connection_string, pool_size=1)

In [7]:
fname = 'feed.xml'

In [8]:
if os.path.isfile(fname):
    os.remove(fname)

In [9]:
url = 'http://dataskeptic.com/feed.rss'
url = 'http://dataskeptic.libsyn.com/rss'

if not(os.path.isfile(fname)):
    print('fetching')
    r = requests.get(url)
    f = open(fname, 'wb')
    f.write(r.text.encode('utf-8'))
    f.close()

with open(fname) as fd:
    xml = xmltodict.parse(fd.read())

fetching


In [10]:
emap = {}
episodes = xml['rss']['channel']['item']
for episode in episodes:
    guid = episode['guid']['#text']
    url = episode['enclosure']['@url']
    emap[guid] = url

In [11]:
len(episodes)

223

In [12]:
q = """
SELECT t1.blog_id, prettyname, guid
 , t2.content_id as mp3_content, t3.content_id as homepage_content
 , t1.title, t1.abstract
FROM blog t1
LEFT JOIN related_content t2
 on t1.blog_id = t2.blog_id
 and t2.type='mp3'
LEFT JOIN related_content t3
 on t1.blog_id = t3.blog_id
 and t3.type='homepage-image'
WHERE t1.prettyname like '/episodes/%%'
AND (t2.blog_id is NULL or t3.blog_id is null)
LIMIT 10
"""
df = pd.read_sql(q, conn)

In [13]:
df.shape

(0, 7)

In [31]:
df

,blog_id,prettyname,guid,mp3_content,homepage_content,title,abstract
0,396,/episodes/2018/fake-news,5b239f1adacc4d51b265d2bcaf572c31,None,None,Fake News,"This episode kicks off our new theme of ""Fake ..."


In [32]:
tpl = """
INSERT INTO related_content (blog_id, dest, type, title, body, duration) VALUES 
({blog_id}, '{dest}', '{type}', '{title}', '{body}', {duration})
"""

In [33]:
for r in range(df.shape[0]):
    row = df.iloc[r]
    blog_id = row['blog_id']
    guid = row['guid']
    title = row['title'].replace("'", "\\'")
    body = row['abstract'].replace("'", "\\'")
    if guid in emap:
        dest = emap[guid]
        response = urllib.request.urlopen(dest)
        data = response.read()
        fname = 'temp.mp3'
        f = open(fname, 'wb')
        f.write(data)
        f.close()
        audio = MP3(fname)
        duration = int(audio.info.length)
        q1 = tpl.format(blog_id=blog_id, dest=dest, type='mp3', title=title, body=body, duration=duration)
        r1 = conn.execute(q1)
        dest = album_art
        q2 = tpl.format(blog_id=blog_id, dest=dest, type='homepage-image', title=title, body=body, duration=-1)
        r2 = conn.execute(q2)

In [34]:
#conn.execute("delete from related_content where blog_id=382")

In [35]:
pd.read_sql("SELECT prettyname from blog order by blog_id desc limit 2", conn).iloc[0]['prettyname']

'/episodes/2018/fake-news'

In [36]:
pd.read_sql("SELECT * from related_content order by content_id desc limit 10", conn)

,content_id,blog_id,dest,type,title,body,created_date,blog_id2,duration
0,696,396,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Fake News,"This episode kicks off our new theme of ""Fake ...",2018-07-13 15:21:40,NaN,-1.0
1,695,396,http://traffic.libsyn.com/dataskeptic/fake-new...,mp3,Fake News,"This episode kicks off our new theme of ""Fake ...",2018-07-13 15:21:39,NaN,2298.0
2,694,396,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Robert Sheaffer,Robert Sheaffer is a freelance writer and skep...,2018-07-13 03:36:16,-1.0,NaN
3,693,396,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Brad Schwartz,"Brad Schwartz is the author of ""Broadcast Hyst...",2018-07-13 03:33:58,-1.0,NaN
4,692,395,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Paige Bailey,Paige Bailey is a senior Cloud Developer Advoc...,2018-07-11 22:53:20,-1.0,NaN
5,691,395,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Damian Brady,Damian Brady is a Cloud Developer Advocate spe...,2018-07-11 22:52:33,-1.0,NaN
6,690,395,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Donovan Brown,Donovan Brown is the Principal DevOps Manager ...,2018-07-11 22:51:17,-1.0,NaN
7,689,395,https://s3.amazonaws.com/dataskeptic.com/img/2...,homepage-image,Dev Ops for Data Science,We revisit the 2018 Microsoft Build in this ep...,2018-07-11 20:26:28,NaN,-1.0
8,688,395,http://traffic.libsyn.com/dataskeptic/devops-f...,mp3,Dev Ops for Data Science,We revisit the 2018 Microsoft Build in this ep...,2018-07-11 20:26:28,NaN,2299.0
9,687,393,https://s3.amazonaws.com/dataskeptic.com/guest...,person,Ramya Ramakrishnan,Ramya Ramakrishnan is a PhD student at MIT in ...,2018-07-09 23:49:52,-1.0,NaN


In [19]:
pd.read_sql("SELECT * from bonus_episodes", conn)

,episode_id,title,link,img,desc,pubDate,enclosure_url,guid,last_updated,blog_id
0,1,Introduction,https://dataskeptic.com,https://s3.amazonaws.com/data-skeptic-bonus-fe...,So what's the deal with the bonus feed? How d...,2017-05-01 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,0b79795d3efc95b9976c7c5b933afce2,2017-05-08 01:24:59,NaN
1,2,"Jill Darling, Survey Director for the Understa...",https://dataskeptic.com/blog/polling/2017/inte...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Featured recently on Data Skeptic, this is the...",2017-05-02 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,7453e2daa86d4e028125f4ee74be97c5,2018-04-01 01:06:14,252.0
2,3,Easily Fooling Deep Neural Networks,https://dataskeptic.com/blog/episodes/2015/eas...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,This episode is from the Data Skeptic archives...,2017-05-12 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,67c7eec88871451c9603d3b361b51e49,2017-05-12 23:42:28,NaN
3,4,"Ernie Tedeschi, Economics, on Poll Re-Weighting",https://dataskeptic.com/blog/polling/2017/inte...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,Our recent episode Opinion Polling for Preside...,2017-05-07 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,4d67d8112ac54a85bea403d842937d37,2018-04-01 01:06:19,254.0
4,5,"David Carmona, GM of AI at Microsoft",https://dataskeptic.com/blog/tools-and-techniq...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Discussion of AI, cognitive services, and brin...",2017-06-15 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,59105f0a4ee011e7be430e0f2d4e9582,2018-04-01 01:05:43,267.0
5,6,"Rohan Kumar, GM for the Database Systems Group...",https://dataskeptic.com/blog/tools-and-techniq...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"Discussion of database as a service, database ...",2017-06-12 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,5bb2dacb4ee011e7be430e0f2d4e9582,2018-04-01 01:05:42,266.0
6,7,Prophecy Fulfillment,https://dataskeptic.com/blog/bonus/prophecy-fu...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,A short recording made in Vietnam,2017-09-12 03:46:50,https://s3.amazonaws.com/data-skeptic-bonus-fe...,02a58e0c976d11e7999d0e0f2d4e9582,2018-04-01 01:03:30,288.0
7,8,Deploying Machine Learning to Production with ...,https://dataskeptic.com/blog/bonus/2018/deploy...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,"In this episode, I discuss the process with To...",2018-01-29 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,5d2718172a354a85beac03d84e93ffab,2018-04-01 01:03:29,333.0
8,10,AI at Microsoft,https://dataskeptic.com/blog/bonus/2018/ai-at-...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,In this episode I catch up with Paige Bailey a...,2018-03-27 14:38:12,https://s3.amazonaws.com/data-skeptic-bonus-fe...,234518172a654affff6c03684e6332f2,2018-04-01 01:03:26,357.0
9,11,Charlie Maloney - Journalist and Loebner Prize...,https://dataskeptic.com/blog/bonus/2018/charli...,https://s3.amazonaws.com/data-skeptic-bonus-fe...,This is the uncut version of my interview with...,2018-04-13 00:00:00,https://s3.amazonaws.com/data-skeptic-bonus-fe...,8172ef3ffa24a8abea714335c07d85da,2018-04-13 21:25:05,364.0


In [26]:
pd.read_sql("""
SELECT t2.guid, t2.title, t1.content_id, t1.blog_id, t1.duration, t3.guid
from related_content t1
JOIN blog t2
 on t1.blog_id = t2.blog_id
LEFT JOIN bonus_episodes t3
 on t1.dest = t3.enclosure_url
WHERE t1.type='mp3'
AND dest like '%%amazonaws%%'
AND (t2.guid is null or t2.guid='' or t2.guid='None')
AND t3.guid is not null
""", conn)

,guid,title,content_id,blog_id,duration,guid


In [27]:
conn.execute("""
UPDATE related_content t1, blog t2, bonus_episodes t3
SET t2.guid = t3.guid
WHERE t1.blog_id = t2.blog_id
 AND t1.dest = t3.enclosure_url
AND t1.type='mp3'
AND dest like '%%amazonaws%%'
AND (t2.guid is null or t2.guid='' or t2.guid='None')
AND t3.guid is not null
""")